# 1. 타겟종이 존재하는 위치 주변의 정보 파악하기  

In [3]:
import pandas as pd
import numpy as np
import os
import pandas as pd
from tqdm import tqdm
import numpy as np
from haversine import haversine

## 1-1. 타겟종 주변의 무당벌레 종 분포 파악

In [3]:
# 무당벌레 전체 관측목록
kellog = pd.read_csv("./brookings_absence후보.csv")

In [36]:
gps =[]
for i in kellog.gps:
    gps.append((float(i.split(',')[0].replace("(","")),float(i.split(',')[1].replace(")",""))))

In [37]:
kellog.gps = gps

In [13]:
clev = []
# 이스트 71번가, Cornelia Ave,이스트 86번가,E 70th St, 케네디에비뉴, 이스트 93번가
clev.append(df[(df.Latitude<=41.527644)&(df.Latitude>=41.463280)&(df.Longitude>=-82)&(df.State=='Ohio')].index)
# 유니온 애비뉴
clev.append(df[(df.Longitude<=-81.585013)&(df.Longitude>=-81.646058)&(df.State=='Ohio')].index)




In [14]:
clev_idx = []
for i in clev:
    for j in i:
        clev_idx.append(j)

In [15]:
kellog = df.iloc[clev_idx].reset_index(drop=True)

In [18]:
kellog = kellog.drop_duplicates(keep='first').reset_index(drop=True)

In [4]:
# 무당벌레 전체 관측목록
df = pd.read_csv("../Data/20220209_07_21_무당벌레관측.csv")



# gps 오차 1000m미만만 남기기
df = df[df.public_positional_accuracy<=1000].reset_index(drop=True)


#Latitude, Longitude 0으로 기록된 부분들 삭제
df = df[(df.Latitude!=0.0)&(df.Longitude!=0.0)].reset_index(drop=True)


#gps 변환
df.Latitude = df.Latitude.astype(float)
df.Longitude = df.Longitude.astype(float)

df_gps = []
for i in tqdm(range(len(df))):
    df_gps.append( (df.Latitude[i], df.Longitude[i]) )
    
    
df['gps'] = df_gps   

#sp.로 들어간 애들을 제거
sp_idx = []
for i in range(len(df)):
    if 'sp.' in df.Species[i]:
        sp_idx.append(i)
df= df.drop(sp_idx).reset_index(drop=True)

# value_counts 해서 통계적으로 유의미한 30개도 안되는 애들 고르기
under_30 = []
for i in range(len(df.Species.value_counts())):
    if df.Species.value_counts()[i]<30:
        under_30.append(df.Species.value_counts().index[i])
        
        
# 고른애들과 이름이 같다면 drop해야함
drop_30 = []
for i in tqdm(range(len(df.Species))):
    for j in under_30:
        if df.Species[i] == j:
            drop_30.append(i)        
df= df.drop(drop_30).reset_index(drop=True)            
            
# 속만 있는 경우 제거
one_letter = []
for i in df.Species.value_counts().index:
    if len(i.split(' ')) == 1:
        one_letter.append(i)    
        
drop_one = []
for i in tqdm(range(len(df.Species))):
    for j in one_letter:
        if df.Species[i] == j:
            drop_one.append(i)        
df = df.drop(drop_one).reset_index(drop=True)

100%|███████████████████████████████████████████████████████████████████████| 175834/175834 [00:13<00:00, 13473.53it/s]


In [19]:
# 타겟종의 데이터프레임에 주변 정보를 카운트해서 넣을 컬럼 생성
for i in df.Species.value_counts().index:

    kellog[i+'_18'] = 0

In [20]:
# 주변 종 체크
for i in tqdm(range(len(kellog))): # target만 뽑아놓은 데이터 프레임
    for j in range(len(df.gps)):  # 전체 무당벌레 데이터 프레임
        if haversine(kellog.gps[i], df.gps[j], unit='km') <= 18 and kellog.Year[i] == df.Year[j]:
            kellog[df.Species[j]+'_18'][i]+=1

  0%|                                                                                          | 0/172 [00:00<?, ?it/s]<ipython-input-20-a2d82a62ea73>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kellog[df.Species[j]+'_18'][i]+=1
100%|████████████████████████████████████████████████████████████████████████████████| 172/172 [08:46<00:00,  3.06s/it]


In [21]:
kellog.iloc[:,10:].describe()

,Harmonia axyridis_18,Coccinella septempunctata_18,Hippodamia convergens_18,Coelophora maculata_18,Cycloneda sanguinea_18,Propylea quatuordecimpunctata_18,Cycloneda munda_18,Psyllobora vigintimaculata_18,Chilocorus stigma_18,Olla v-nigrum_18,...,Nephus flavifrons_18,Brachiacantha quadripunctata_18,Hyperaspis proba_18,Egius platycephalus_18,Hyperaspis connectens_18,Nephus intrusus_18,Scymnus louisianae_18,Exochomus fasciatus_18,Coccinella undecimpunctata_18,Delphastus pusillus_18
count,172.000000,172.000000,172.000000,172.000000,172.0,172.000000,172.000000,172.000000,172.000000,172.000000,...,172.0,172.0,172.0,172.0,172.0,172.0,172.0,172.0,172.0,172.0
mean,19.720930,1.453488,0.011628,2.837209,0.0,1.883721,0.906977,0.191860,0.220930,0.250000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
std,12.575062,1.724654,0.107517,2.521767,0.0,1.875784,1.155995,0.394914,0.416085,0.434277,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25%,11.000000,0.000000,0.000000,1.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50%,17.000000,1.000000,0.000000,2.000000,0.0,2.000000,1.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75%,33.500000,3.000000,0.000000,4.000000,0.0,3.000000,1.000000,0.000000,0.000000,0.250000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
max,46.000000,8.000000,1.000000,9.000000,0.0,9.000000,4.000000,1.000000,1.000000,1.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### 1-2. 타겟종 주변의 새 정보 파악

In [22]:
os.chdir(path = '../Data/E_bird/')

In [27]:
# Ebird에서 가져온 새 목록 합치기


# directory 내의 csv들을 전부 가져와서
for i in range(len(os.listdir())):
    globals()['bird{}'.format(i)] = pd.read_csv(os.listdir()[i],sep='\t')

# for_merge 라는 변수에 모두 합쳐놓음
for_merge = bird1.iloc[0:0]
for i in range(26):
    for_merge = pd.concat([for_merge,globals()['bird{}'.format(i)]])
bird2021 = pd.read_csv("../새2021.csv")
for_merge = pd.concat([for_merge, bird2021]).reset_index(drop=True)


# EBIRD자료는 GPS정확도를 제공하지 않으므로, GPS정확도가 존재한다면 INAT의 자료임
for_merge = for_merge[for_merge.coordinateUncertaintyInMeters.isnull()].reset_index(drop=True)

# 필요한 컬럼들만 남김
for_merge=for_merge[['species','year','month','decimalLatitude','decimalLongitude']].reset_index(drop=True)

# 종 이름이 NAN이면 쓸수 없으므로 DROP
for_merge=for_merge.drop(for_merge[for_merge.species.isnull()==True].index).reset_index(drop=True)

# haversine은 set에 들어있는 gps값을 받음 (lat, lon) 이형식으로 변환하는 과정
for_gps = []
for i in tqdm(range(len(for_merge))):
    for_gps.append( (for_merge['decimalLatitude'][i], for_merge['decimalLongitude'][i]) )
    
for_merge['gps'] = for_gps



for_merge=for_merge[(for_merge.species!='Locustella certhiola')&(for_merge.species!='Locustella ochotensis')&
         (for_merge.species!='Ianthocincla pectoralis')&(for_merge.species!='Luscinia calliope')&
         (for_merge.species!='Iduna aedon')&(for_merge.species!='Phylloscopus sibillatrix')&
         (for_merge.species!='Muscicapa latirostris')&(for_merge.species!='Tiaris bicolor')&
         (for_merge.species!='Tiaris canorus')&(for_merge.species!='Lonchura oryzivora')&
         (for_merge.species!='Lonchura cucullata')&(for_merge.species!='Estrilda coerulescens')&
         (for_merge.species!='alcarius ornatus')]

# 무당벌레는 관측지역에 잡힌 숫자를 믿을 수 없어서 관측 자체만 믿었음, 새 역시도 drop_duplicate로 동일한 절차 진행
for_merge = for_merge.drop_duplicates(keep='first').reset_index(drop=True)


# 타겟종의 데이터프레임에 주변 정보를 카운트해서 넣을 컬럼 생성
for i in for_merge.species.value_counts().index:
    kellog[i+'_18'] = 0

100%|████████████████████████████████████████████████████████████████████| 3034469/3034469 [00:27<00:00, 111420.85it/s]


In [28]:
# 타겟종의 데이터프레임에 주변 정보를 카운트해서 넣을 컬럼 생성
for i in for_merge.species.value_counts().index:
    kellog[i+'_18'] = 0

In [29]:
for i in tqdm(range(len(kellog))): # 타겟종만 뽑아놓은 데이터 프레임
    for j in range(len(for_merge.gps)):  # 전체 새 데이터 프레임
            if haversine(kellog.gps[i], for_merge.gps[j], unit='km') <= 18 and kellog.Year[i] == for_merge.year[j]:
                # 타겟종 관측 좌표와 새 관측좌표의 거리가 1km이하이고, 해당 좌표의 관측 연도가 같다면 count +1
                kellog[for_merge.species[j]+'_18'][i]+=1


  0%|                                                                                          | 0/172 [00:00<?, ?it/s]<ipython-input-29-5a383200347b>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kellog[for_merge.species[j]+'_18'][i]+=1
100%|██████████████████████████████████████████████████████████████████████████████| 172/172 [1:22:02<00:00, 28.62s/it]


In [30]:
kellog.to_csv("./clev_absence_조사.csv")

In [36]:
kellog

,Unnamed: 0,Unnamed: 0.1,Source,Year,Species,Latitude,Longitude,public_positional_accuracy,State,gps,...,Acrocephalus dumetorum_18,Cyanerpes cyaneus_18,Acrocephalus schoenobaenus_18,Pachyramphus major_18,Luscinia cyane_18,Taeniopygia guttata_18,Locustella fluviatilis_18,Oenanthe pleschanka_18,Phoenicurus phoenicurus_18,Lanius collurio_18
0,1262,1262,The Lost Ladybugs,2008,Harmonia axyridis,41.475603,-81.839303,0.0,Ohio,"(41.4756031, -81.839303)",...,0,0,0,0,0,0,0,0,0,0
1,4179,4179,I-naturalist,2009,Propylea quatuordecimpunctata,41.500688,-81.108375,0.0,Ohio,"(41.500688, -81.108375)",...,0,0,0,0,0,0,0,0,0,0
2,4217,4217,The Lost Ladybugs,2009,Harmonia axyridis,41.481993,-81.798190,0.0,Ohio,"(41.4819932, -81.79819)",...,0,0,0,0,0,0,0,0,0,0
3,4224,4224,The Lost Ladybugs,2009,Coccinella septempunctata,41.481993,-81.798190,0.0,Ohio,"(41.4819932, -81.79819)",...,0,0,0,0,0,0,0,0,0,0
4,4227,4227,The Lost Ladybugs,2009,Harmonia axyridis,41.520052,-81.556235,0.0,Ohio,"(41.5200518, -81.556235)",...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,191973,1591,INAT,2021,Harmonia axyridis,41.202309,-81.645637,32.0,Ohio,"(41.2023089, -81.6456369)",...,0,0,0,0,0,0,0,0,0,0
168,195764,5382,INAT,2021,Propylea quatuordecimpunctata,41.541844,-81.629481,6.0,Ohio,"(41.54184426, -81.62948121)",...,0,0,0,0,0,0,0,0,0,0
169,196679,6297,INAT,2021,Harmonia axyridis,41.542319,-81.629588,136.0,Ohio,"(41.5423186382, -81.6295879655)",...,0,0,0,0,0,0,0,0,0,0
170,197356,6974,INAT,2021,Cycloneda munda,41.326275,-81.643792,30.0,Ohio,"(41.326275, -81.6437916667)",...,0,0,0,0,0,0,0,0,0,0


In [37]:
kellog[kellog['Adalia bipunctata_18']==0]

,Unnamed: 0,Unnamed: 0.1,Source,Year,Species,Latitude,Longitude,public_positional_accuracy,State,gps,...,Acrocephalus dumetorum_18,Cyanerpes cyaneus_18,Acrocephalus schoenobaenus_18,Pachyramphus major_18,Luscinia cyane_18,Taeniopygia guttata_18,Locustella fluviatilis_18,Oenanthe pleschanka_18,Phoenicurus phoenicurus_18,Lanius collurio_18
0,1262,1262,The Lost Ladybugs,2008,Harmonia axyridis,41.475603,-81.839303,0.0,Ohio,"(41.4756031, -81.839303)",...,0,0,0,0,0,0,0,0,0,0
1,4179,4179,I-naturalist,2009,Propylea quatuordecimpunctata,41.500688,-81.108375,0.0,Ohio,"(41.500688, -81.108375)",...,0,0,0,0,0,0,0,0,0,0
2,4217,4217,The Lost Ladybugs,2009,Harmonia axyridis,41.481993,-81.798190,0.0,Ohio,"(41.4819932, -81.79819)",...,0,0,0,0,0,0,0,0,0,0
3,4224,4224,The Lost Ladybugs,2009,Coccinella septempunctata,41.481993,-81.798190,0.0,Ohio,"(41.4819932, -81.79819)",...,0,0,0,0,0,0,0,0,0,0
4,4227,4227,The Lost Ladybugs,2009,Harmonia axyridis,41.520052,-81.556235,0.0,Ohio,"(41.5200518, -81.556235)",...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,191973,1591,INAT,2021,Harmonia axyridis,41.202309,-81.645637,32.0,Ohio,"(41.2023089, -81.6456369)",...,0,0,0,0,0,0,0,0,0,0
168,195764,5382,INAT,2021,Propylea quatuordecimpunctata,41.541844,-81.629481,6.0,Ohio,"(41.54184426, -81.62948121)",...,0,0,0,0,0,0,0,0,0,0
169,196679,6297,INAT,2021,Harmonia axyridis,41.542319,-81.629588,136.0,Ohio,"(41.5423186382, -81.6295879655)",...,0,0,0,0,0,0,0,0,0,0
170,197356,6974,INAT,2021,Cycloneda munda,41.326275,-81.643792,30.0,Ohio,"(41.326275, -81.6437916667)",...,0,0,0,0,0,0,0,0,0,0
